In [14]:
from importlib import reload ,import_module
import module.utilize as utilize
import module.singleVariant as singleVariant
import module.multiVariant as multiVariant
import numpy as np
from numba import njit,jit, float32
from tqdm import tqdm
import module.multiHistogramSparse as multiHistogramSparse
import module.ananlysisFuncion as analysisFunction
import module.singleVariantCopulaBase as CopulaBase
reload(multiHistogramSparse)
reload(utilize)
reload(multiVariant)
reload(singleVariant)
reload(analysisFunction)
reload(CopulaBase)

#attribute_names=np.array(["SALT","TEMP","U","V","W"])
attribute_names=np.array(["phi_grav","particle_mass_density","zmom","ymom","xmom"])
attribute_names=np.array(["phi_grav","particle_mass_density","zmom","ymom","xmom"])
incremental_number=300
all_ensamble_data=np.empty([0,incremental_number,64,64,64])

for name in attribute_names:
    data=utilize.readFiles(name,incremental_number)
    data=data.reshape(1,incremental_number,64,64,64)
    all_ensamble_data=np.append(all_ensamble_data,data,axis=0)



covBlockSize=5
dataBlockSize=5
binsNumber=128
sizeZ=60
sizeY=60
sizeX=60
minMaxBlockSize=2
isMinMax=False

data=all_ensamble_data[:,:, 0:sizeZ, 0:sizeY, 0:sizeX]
conditions=np.array([[36,40],[26,30]])

print("start fit model")
multiHistBlockSize=3
multiHistModel=multiHistogramSparse.multiHistogramSpaseModel(data,blockSize=multiHistBlockSize,binsNumber=binsNumber)
multiHistModel.fit()
print("multi-hist complete fit")
gtModel=multiHistogramSparse.multiHistogramSpaseModel(data,blockSize=1,binsNumber=binsNumber)
gtModel.fit()
print("gt model complete")
multiHistModel.getStorageSize()

start fit model
multi-hist complete fit
gt model complete
sparse sum: 5966862


5966862

In [15]:
mtError=[]
with tqdm(total=sizeZ*sizeY*sizeX, desc="總進度") as pbar:
    for idx in range(sizeZ * sizeY * sizeX):
        
        z = idx // (sizeY * sizeX)
        y = (idx // sizeX) % sizeY
        x = idx % sizeX        
        ### GroundTruth ###

        gtMultiHistModel=gtModel.getHistByPos(z,y,x)

        ### Mt model
        mtMultiHistModel=multiHistModel.getHistByPos(z,y,x)
        
        #print(f"multi hist hist 執行時間:{processHistTimeEnd-ProcessHistTime}")
        
        rmse=multiHistogramSparse.rmseForSparseHistogram(gtMultiHistModel,mtMultiHistModel)
        
        #print(f"multi hist RMSE 執行時間:{processRmseTimeEnd-processRmseTime}")
        mtError.append(rmse)

        #print(f"multi hist 執行時間:{ProcessTimeEnd-ProcessTime}")
        
        pbar.update(1)

mtError=np.array(mtError)
mtError=mtError.mean()

print(mtError)

總進度: 100%|██████████| 216000/216000 [00:17<00:00, 12506.75it/s]

0.022730927579120923
